In [1]:
import requests
import re
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup 
from tqdm import tqdm
from urllib.request import urlopen

In [3]:
headers = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'}

driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(1)

In [4]:
url = 'https://elaw.klri.re.kr/kor_service/lawNameList.do'
driver.get(url)

In [6]:
html = driver.page_source
page_html = BeautifulSoup(html,'html.parser')
type(page_html)

bs4.BeautifulSoup

In [8]:
# 법령 검색 화면에서의 페이지 xpath

x_path_base = '//*[@id="searchListForm"]/section/div[2]/a[{}]{}'
x_path = []

for i in range(3,13):
    if i == 12 :
        x_path.append(x_path_base.format(i,'/img'))
    else :
        x_path.append(x_path_base.format(i,''))
for each in x_path:
    print(each)

//*[@id="searchListForm"]/section/div[2]/a[3]
//*[@id="searchListForm"]/section/div[2]/a[4]
//*[@id="searchListForm"]/section/div[2]/a[5]
//*[@id="searchListForm"]/section/div[2]/a[6]
//*[@id="searchListForm"]/section/div[2]/a[7]
//*[@id="searchListForm"]/section/div[2]/a[8]
//*[@id="searchListForm"]/section/div[2]/a[9]
//*[@id="searchListForm"]/section/div[2]/a[10]
//*[@id="searchListForm"]/section/div[2]/a[11]
//*[@id="searchListForm"]/section/div[2]/a[12]/img


In [9]:
page_nums = []
try :
    for j in tqdm(range(21)) :
        for x_path_ in x_path:
            html = driver.page_source
            page_html = BeautifulSoup(html,'html.parser')
            
            # 각 법령의 페이지 번호
            for each in page_html.select('#searchListForm > section > table > tbody > tr > td > a'):
                page_nums.append(each.attrs['href'][53:58])
            driver.find_element_by_xpath(x_path_).click()
    page_nums = set(page_nums)

except Exception as e :
    print(e)
    print(j)
    print(each)
    print(len(page_nums))

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [01:10<00:00,  3.36s/it]


In [10]:
page_nums = [each for each in page_nums]
page_nums

['46875',
 '42822',
 '48487',
 '38383',
 '46489',
 '41228',
 '48157',
 '46671',
 '48658',
 '48448',
 '46391',
 '48730',
 '46187',
 '47127',
 '48059',
 '44549',
 '23641',
 '33696',
 '48974',
 '45672',
 '42201',
 '39573',
 '47216',
 '46204',
 '43574',
 '46329',
 '43955',
 '40247',
 '49104',
 '43944',
 '45262',
 '49561',
 '46966',
 '45828',
 '48580',
 '43901',
 '44831',
 '39547',
 '40185',
 '48656',
 '48495',
 '50512',
 '18314',
 '46970',
 '47725',
 '46628',
 '47913',
 '48710',
 '42585',
 '45759',
 '48158',
 '35748',
 '45597',
 '45620',
 '46384',
 '10508',
 '48918',
 '46342',
 '45763',
 '46777',
 '45708',
 '24346',
 '45734',
 '40240',
 '29385',
 '47397',
 '48624',
 '42654',
 '47393',
 '42814',
 '37510',
 '40926',
 '44535',
 '45722',
 '45056',
 '46487',
 '31096',
 '37681',
 '37888',
 '46327',
 '49291',
 '45862',
 '48186',
 '46655',
 '45912',
 '48931',
 '38607',
 '48653',
 '49108',
 '18435',
 '45346',
 '46593',
 '42761',
 '42745',
 '46751',
 '50164',
 '44781',
 '49642',
 '43706',
 '45728',


---

# 루프 돌며 데이터 스크래핑

In [93]:
base_url = 'http://elaw.klri.re.kr/eng_service/lawView.do?hseq={}&lang=KOR'
    
for idx, num in tqdm(enumerate(page_nums)) :
    try :
        url = base_url.format(num)

        driver.get(url)
#         time.sleep(0.1)

        html = driver.page_source
        page_html = BeautifulSoup(html,'html.parser')

        # 스크래핑을 위한 페이지 HTML
        src = str(page_html.find_all('iframe')[1].get('src'))
        url_base = 'http://elaw.klri.re.kr'
        new_url = url_base + src

        resp = urlopen(new_url)
        soup = BeautifulSoup(resp,'html.parser')


        html_str = str(soup)
        for cl in ('"section"', '"subsection"', '"paragraph"', '"subparagraph"', '"item"', 
                   '"part"', '"article"', '"addenda"', '"addenda_none"' ,'"chapter"', '"articletitle"',
                   '"none"', '"ho"', '"hang"') :
            html_str = re.sub(cl, 'scrap_data', html_str)
        soup = BeautifulSoup(html_str)


        law_ENG = []
        law_KOR = []

        # 법률 제목

        law_title = soup.select('.lawmultiview > table > tr')[0].select('td')
        law_title_ENG = law_title[0].getText().replace('\r','').replace('\n','').replace('\t','')
        law_title_KOR = law_title[1].getText().replace('\r','').replace('\n','').replace('\t','')

        law_ENG.append(law_title_ENG)
        law_KOR.append(law_title_KOR)

        # 관련 법률

        law_info = soup.select('.lawmultiview > table > tr')[1].select('td')
        law_info_ENG = re.sub('\n+','\n', law_info[0].getText().strip().replace('\t','').replace('\r',''))
        law_info_KOR = re.sub('\n+','\n', law_info[1].getText().strip().replace('\t','').replace('\r',''))

        law_ENG.append(law_info_ENG)
        law_KOR.append(law_info_KOR)

        # 각 데이터 삽입
        for row in soup.select('.lawmultiview > table > tr')[2:] :
            eng_tmp = ''
            kor_tmp = ''

            for each_eng in row.find('td').select('.scrap_data') :
                eng_tmp += each_eng.getText() + '\n'
            law_ENG.append(eng_tmp.strip())    

            for each_kor in row.find('td').find_next_sibling('td').select('.scrap_data') :
                kor_tmp += each_kor.getText() + '\n'
            law_KOR.append(kor_tmp.strip())

        df = pd.DataFrame(list(zip(law_ENG, law_KOR)), columns =['Eng_Rough_Ver', 'Kor_Rough_Ver']) 
        df.to_excel('./scrapped/{:0>4d}_{}_190722.xlsx'.format(idx, df.iloc[0].Kor_Rough_Ver))
    
    except Exception as e :
        print(e)
        continue





0it [00:00, ?it/s]


1it [00:01,  1.80s/it]


2it [00:04,  2.13s/it]


3it [00:08,  2.52s/it]


4it [00:12,  3.02s/it]


5it [00:15,  3.01s/it]


6it [00:17,  2.79s/it]


7it [00:20,  2.80s/it]


8it [00:23,  2.87s/it]


9it [00:26,  3.05s/it]


10it [00:30,  3.15s/it]


11it [00:32,  2.77s/it]


12it [00:34,  2.54s/it]


13it [00:38,  3.00s/it]


14it [00:40,  2.78s/it]


15it [00:44,  3.12s/it]


16it [00:49,  3.67s/it]


17it [00:51,  3.18s/it]


18it [00:54,  3.14s/it]


19it [01:24, 11.12s/it]


20it [01:27,  8.73s/it]


21it [01:30,  7.02s/it]


22it [01:32,  5.51s/it]


23it [01:34,  4.43s/it]


24it [01:36,  3.63s/it]


25it [01:38,  3.18s/it]


26it [01:42,  3.39s/it]


27it [01:44,  3.21s/it]


28it [01:47,  3.04s/it]

HTTP Error 500: Internal Server Error





29it [01:48,  2.56s/it]


30it [01:50,  2.36s/it]


31it [01:53,  2.33s/it]


32it [01:55,  2.35s/it]


33it [01:58,  2.52s/it]


34it [02:00,  2.32s/it]


35it [02:02,  2.28s/it]


36it [02:04,  2.13s/it]


37it [02:06,  2.10s/it]


38it [02:08,  2.06s/it]


39it [02:10,  2.11s/it]


40it [02:14,  2.67s/it]


41it [02:17,  2.70s/it]


42it [02:19,  2.50s/it]


43it [02:23,  2.96s/it]


44it [02:25,  2.65s/it]


45it [02:28,  2.75s/it]


46it [02:30,  2.64s/it]


47it [02:35,  3.37s/it]


48it [02:39,  3.55s/it]


49it [02:41,  3.20s/it]


50it [02:45,  3.37s/it]


51it [02:47,  2.98s/it]


52it [02:49,  2.65s/it]


53it [02:52,  2.68s/it]


54it [02:56,  2.95s/it]


55it [02:58,  2.78s/it]


56it [03:00,  2.70s/it]


57it [03:02,  2.41s/it]


58it [03:05,  2.67s/it]


59it [03:07,  2.44s/it]


60it [03:11,  2.74s/it]


61it [03:13,  2.55s/it]


62it [03:15,  2.37s/it]


63it [03:17,  2.38s/it]


64it [03:19,  2.18s/it]


65it [03:21,  2.24s/it]


66it [03:24,  2.34s/it]


67it [03:

335it [16:38,  2.39s/it]


336it [16:40,  2.44s/it]


337it [16:42,  2.29s/it]


338it [16:44,  2.27s/it]


339it [16:47,  2.29s/it]


340it [16:49,  2.33s/it]


341it [16:51,  2.31s/it]


342it [16:53,  2.13s/it]


343it [16:56,  2.38s/it]


344it [17:00,  2.78s/it]


345it [17:02,  2.53s/it]


346it [17:04,  2.41s/it]


347it [17:06,  2.23s/it]


348it [17:09,  2.56s/it]


349it [17:11,  2.42s/it]


350it [17:13,  2.28s/it]


351it [17:15,  2.28s/it]


352it [17:18,  2.32s/it]


353it [17:20,  2.28s/it]


354it [17:22,  2.22s/it]


355it [17:26,  2.82s/it]


356it [17:29,  2.86s/it]


357it [17:32,  3.01s/it]


358it [17:36,  3.07s/it]


359it [17:38,  2.92s/it]


360it [17:40,  2.71s/it]


361it [17:44,  2.95s/it]


362it [17:48,  3.17s/it]


363it [17:51,  3.10s/it]


364it [17:53,  2.78s/it]


365it [17:55,  2.80s/it]


366it [17:57,  2.53s/it]


367it [18:01,  2.75s/it]


368it [18:03,  2.54s/it]


369it [18:05,  2.54s/it]


370it [18:09,  2.97s/it]


371it [18:11,  2.70s/it]


3

941it [45:18,  2.50s/it]


942it [45:19,  2.29s/it]


943it [45:21,  2.10s/it]


944it [45:24,  2.47s/it]


945it [45:27,  2.56s/it]


946it [45:30,  2.67s/it]


947it [45:32,  2.42s/it]


948it [45:34,  2.36s/it]


949it [45:37,  2.39s/it]


950it [45:39,  2.49s/it]


951it [45:42,  2.38s/it]


952it [45:45,  2.66s/it]


953it [45:49,  3.02s/it]


954it [45:51,  2.83s/it]


955it [45:57,  3.72s/it]


956it [46:00,  3.41s/it]


957it [46:02,  3.01s/it]


958it [46:04,  2.76s/it]


959it [46:08,  3.30s/it]


960it [46:11,  3.01s/it]


961it [46:13,  2.79s/it]


962it [46:15,  2.59s/it]


963it [46:18,  2.82s/it]


964it [46:21,  2.65s/it]


965it [46:22,  2.39s/it]


966it [46:25,  2.30s/it]


967it [46:28,  2.50s/it]


968it [46:32,  3.00s/it]


969it [46:34,  2.83s/it]


970it [46:36,  2.52s/it]


971it [46:38,  2.34s/it]


972it [46:41,  2.65s/it]


973it [46:43,  2.44s/it]


974it [46:47,  2.94s/it]


975it [46:49,  2.62s/it]


976it [46:52,  2.77s/it]


977it [46:55,  2.62s/it]


9

HTTP Error 500: Internal Server Error





1337it [1:05:52,  2.41s/it]


1338it [1:05:54,  2.31s/it]


1339it [1:05:56,  2.32s/it]


1340it [1:05:59,  2.50s/it]


1341it [1:06:01,  2.40s/it]


1342it [1:06:05,  2.67s/it]


1343it [1:06:07,  2.64s/it]


1344it [1:06:11,  2.96s/it]


1345it [1:06:13,  2.77s/it]


1346it [1:06:16,  2.76s/it]


1347it [1:06:18,  2.55s/it]


1348it [1:06:20,  2.50s/it]


1349it [1:06:25,  3.08s/it]


1350it [1:06:27,  2.77s/it]


1351it [1:06:29,  2.50s/it]


1352it [1:06:34,  3.22s/it]


1353it [1:06:36,  3.01s/it]


1354it [1:06:38,  2.71s/it]


1355it [1:06:41,  2.65s/it]


1356it [1:06:43,  2.55s/it]


1357it [1:06:45,  2.46s/it]


1358it [1:06:47,  2.32s/it]


1359it [1:06:50,  2.32s/it]


1360it [1:06:52,  2.41s/it]


1361it [1:07:08,  6.44s/it]


1362it [1:07:11,  5.36s/it]


1363it [1:07:13,  4.38s/it]


1364it [1:07:16,  3.91s/it]


1365it [1:07:19,  3.67s/it]


1366it [1:07:22,  3.60s/it]


1367it [1:07:24,  3.16s/it]


1368it [1:07:27,  3.11s/it]


1369it [1:07:30,  3.08s/it]


1370it 

1610it [1:19:29,  2.55s/it]


1611it [1:19:31,  2.41s/it]


1612it [1:19:34,  2.61s/it]


1613it [1:19:38,  2.84s/it]


1614it [1:19:40,  2.70s/it]


1615it [1:19:42,  2.55s/it]


1616it [1:19:44,  2.41s/it]


1617it [1:19:50,  3.22s/it]


1618it [1:19:53,  3.27s/it]


1619it [1:19:57,  3.47s/it]


1620it [1:19:59,  3.00s/it]


1621it [1:20:02,  2.92s/it]


1622it [1:20:05,  3.06s/it]


1623it [1:20:13,  4.49s/it]


1624it [1:20:15,  3.73s/it]


1625it [1:20:18,  3.60s/it]


1626it [1:20:21,  3.38s/it]


1627it [1:20:23,  3.05s/it]


1628it [1:20:30,  4.29s/it]


1629it [1:20:32,  3.63s/it]


1630it [1:20:35,  3.36s/it]


1631it [1:20:40,  3.80s/it]


1632it [1:20:42,  3.33s/it]


1633it [1:20:45,  3.26s/it]


1634it [1:20:49,  3.46s/it]


1635it [1:20:53,  3.65s/it]


1636it [1:20:58,  3.91s/it]


1637it [1:21:01,  3.60s/it]


1638it [1:21:02,  3.04s/it]


1639it [1:21:04,  2.66s/it]


1640it [1:21:07,  2.73s/it]


1641it [1:21:10,  2.82s/it]


1642it [1:21:13,  2.67s/it]


1643it [1:

---